In [2]:
import os

import pandas as pd
import polars as pl

import kaggle_evaluation.jane_street_inference_server

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDRegressor, LinearRegression

# Load dataset (assuming already combined from multiple parquet files)
df = pd.read_parquet('/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=0/part-0.parquet')
df.describe()

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
count,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06,0.0,0.0,0.0,0.0,0.0,1.944210e+06,...,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06
mean,9.384629e+01,4.240000e+02,1.376638e+01,1.973281e+00,NaN,NaN,NaN,NaN,NaN,-4.463175e-02,...,-9.805073e-02,8.424639e-03,1.076465e-02,2.412764e-03,1.139089e-02,2.185480e-02,3.326982e-03,1.487634e-03,-4.817980e-04,1.078154e-03
std,4.813196e+01,2.450851e+02,1.108778e+01,9.691969e-01,NaN,NaN,NaN,NaN,NaN,9.479579e-01,...,6.398993e-01,9.558780e-01,1.141793e+00,8.442027e-01,1.276044e+00,1.266448e+00,1.225165e+00,8.705768e-01,8.918150e-01,8.737320e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,4.405696e-01,NaN,NaN,NaN,NaN,NaN,-1.176608e+01,...,-3.393299e+00,-5.000000e+00,-5.000000e+00,-5.000000e+00,-5.000000e+00,-5.000000e+00,-5.000000e+00,-5.000000e+00,-5.000000e+00,-5.000000e+00
25%,5.400000e+01,2.120000e+02,7.000000e+00,1.323803e+00,NaN,NaN,NaN,NaN,NaN,-4.756072e-01,...,-3.195696e-01,-2.322211e-01,-2.667868e-01,-1.210319e-01,-4.444537e-01,-5.146699e-01,-2.569897e-01,-3.558709e-01,-3.914038e-01,-2.892584e-01
50%,9.900000e+01,4.240000e+02,1.200000e+01,1.763827e+00,NaN,NaN,NaN,NaN,NaN,-5.818180e-02,...,-2.468792e-01,-3.943805e-03,-2.333469e-02,-1.248489e-03,-1.031224e-02,-1.836913e-02,-4.690550e-03,-9.597129e-03,-2.376243e-02,-3.107830e-04
75%,1.360000e+02,6.360000e+02,1.700000e+01,2.393846e+00,NaN,NaN,NaN,NaN,NaN,3.493771e-01,...,-1.256560e-01,2.309678e-01,2.538152e-01,1.191232e-01,4.292141e-01,5.065366e-01,2.438874e-01,3.360999e-01,3.463979e-01,2.840819e-01
max,1.690000e+02,8.480000e+02,3.800000e+01,6.011999e+00,NaN,NaN,NaN,NaN,NaN,1.532000e+01,...,4.370195e+01,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00


# Data Preprocessing

In [11]:
# Target and features
target = 'responder_6'
features = ['feature_05','feature_06','feature_07','feature_09','feature_10','feature_11','feature_12','feature_13','feature_14','feature_20','feature_22','feature_23','feature_24','feature_25','feature_28','feature_29','feature_30','feature_34','feature_35','feature_36','feature_38','feature_48','feature_49','feature_59','feature_60','feature_61','feature_67','feature_68','feature_69','feature_70','feature_71','feature_72']
# features = [col for col in df.columns if col not in [target, 'date_id','time_id', 'weight', 'symbol_id', 'responder_0', 'responder_1','responder_2','responder_3','responder_4','responder_5','responder_7', 'responder_8', 'responder_9']]
usable_features = df[[col for col in df.columns if col in features]]

scaler = StandardScaler()
usable_features_scaled = scaler.fit_transform(usable_features)

encoded = pd.get_dummies(df['symbol_id'], prefix='symbol_id')
encoded.head()

# # Splitting data into train and test set

# Define a split point
split_index = int(len(df) * 0.8)

# filtered_df = pd.concat([df[['date_id', 'time_id', 'weight']],pd.DataFrame(usable_features_scaled, columns=features), encoded], axis=1)
filtered_df = pd.concat([pd.DataFrame(usable_features_scaled, columns=features), encoded], axis=1)

# Split data
train_data = filtered_df.iloc[:split_index]
test_data = filtered_df.iloc[split_index:]

responder_train = df.iloc[:split_index]
responder_test = df.iloc[split_index:]

# Extract features and target
# X_train = train_data[['symbol_id', 'date_id', 'time_id', 'weight'] + features]
# y_train = responder_train[target]

# X_test = test_data[['symbol_id', 'date_id', 'time_id', 'weight'] + features]
# y_test = responder_test[target]

X_train = train_data
y_train = responder_train[target]

X_test = test_data
y_test = responder_test[target]

# Training basic SGD and BGD models

In [18]:

# SGD and BGD pipelines
# sgd_pipeline = Pipeline(steps=[
#     ('regressor', SGDRegressor(max_iter=1000, tol=1e-3, random_state=42))
# ])

# bgd_pipeline = Pipeline(steps=[
#     ('regressor', LinearRegression())
# ])

# Train models
# sgd_pipeline.fit(X_train, y_train)
# bgd_pipeline.fit(X_train, y_train)

regr = LinearRegression() 
regr.fit(X_train, y_train)

accuracy = regr.score(X_test, y_test)
print(f'Linear Regression Accuracy: {accuracy * 100:.2f}%')

Linear Regression Accuracy: 1.51%


# Attempting to refine SGD using GridSearchCV

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV

param_grid = {
    'alpha': [0.0001, 0.001, 0.01],  # L2 regularization strength
    'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
    'eta0': [0.01, 0.1, 1.0],  # Initial learning rate
}
grid = GridSearchCV(SGDRegressor(max_iter=1000, tol=1e-3), param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid.fit(X_train, y_train)
print(grid.best_params_)


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_stocha

# Support Vector Machine (SVM) model testing

In [ ]:
from sklearn.svm import SVC

svm_model = SVC(kernel='rbf')
svm_model.fit(X_train_scaled_Z, y_train)
svm_accuracy = svm_model.score(X_test_scaled_Z, y_test)
print(f'SVM Accuracy: {svm_accuracy * 100:.2f}%')

The evaluation API requires that you set up a server which will respond to inference requests. We have already defined the server; you just need write the predict function. When we evaluate your submission on the hidden test set the client defined in `jane_street_gateway` will run in a different container with direct access to the hidden test set and hand off the data timestep by timestep.



Your code will always have access to the published copies of the files.

In [ ]:
lags_ : pl.DataFrame | None = None


# Replace this function with your inference code.
# You can return either a Pandas or Polars dataframe, though Polars is recommended.
# Each batch of predictions (except the very first) must be returned within 1 minute of the batch features being provided.
def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    # All the responders from the previous day are passed in at time_id == 0. We save them in a global variable for access at every time_id.
    # Use them as extra features, if you like.
    global lags_
    if lags is not None:
        lags_ = lags

    # Replace this section with your own predictions
    predictions = test.select(
        'row_id',
        pl.lit(0.0).alias('responder_6'),
    )

    if isinstance(predictions, pl.DataFrame):
        assert predictions.columns == ['row_id', 'responder_6']
    elif isinstance(predictions, pd.DataFrame):
        assert (predictions.columns == ['row_id', 'responder_6']).all()
    else:
        raise TypeError('The predict function must return a DataFrame')
    # Confirm has as many rows as the test data.
    assert len(predictions) == len(test)

    return predictions

When your notebook is run on the hidden test set, inference_server.serve must be called within 15 minutes of the notebook starting or the gateway will throw an error. If you need more than 15 minutes to load your model you can do so during the very first `predict` call, which does not have the usual 1 minute response deadline.

In [ ]:
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet',
            '/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet',
        )
    )